In [2]:
# Imports
import csv
import pandas as pd
import nltk
import re
import collections
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('rslp')
import matplotlib.pyplot as plt
import heapq
import time

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [0]:
# Lendo o CSV
colecao = pd.read_csv('https://raw.githubusercontent.com/LDVictor/ri_lab_01/master/output/results.csv')
colecao = colecao.astype(str)
documentos = colecao['texto']

# Laboratório 04: Indexação e Processamento de Consulta

# Questão 1

Execute o algoritmo ilustrado na Fig. 5.8 do livro texto (pag. 157) para gerar um índice similar o mostrado na Fig. 5.4 (pag. 134). Guarde o índice em disco em formato csv.

In [0]:
stopwords = stopwords.words("portuguese")
tokenizador = RegexpTokenizer(r'\b[A-zÀ-ú\d\-\']+')
indices = {}
n = 0

for documento in documentos:
    n += 1
    tokens = tokenizador.tokenize(documento.lower())
    for token in set(tokens):
        if (token not in stopwords and len(token) >= 2):
            contador = (n, tokens.count(token))
            if (token not in indices.keys()):
                  indices[token] = []
                  indices[token].append(contador)
            else:
                  indices[token].append(contador)

In [0]:
# Guardando o indice em formato CSV
inverted_index_df = pd.DataFrame.from_dict(indices, orient="index")
inverted_index_df.to_csv('data.csv')
indices = indices

# Questão 2

Implemente as abordagens de processamento de consulta documento-por-vez e termo-por-vez.


1.   Defina 5 consultas de um termo somente.
2.   Execute as 5 consultas em cada algoritmo retornando os top-10 documentos (parâmetro k do algoritmo).
3.   Dê evidências de que sua implementação está correta.
4.   Compare os tempos médios de execução e uso de memória de cada algoritmo.

In [7]:
# Funcao que implementa a consulta documento-por-vez
def documentoPorVez(consulta, indices, k):
    listas_invertidas = []
    r = []
    for palavra in consulta.split(" "):
        if palavra in indices.keys():
            listas_invertidas.append(indices[palavra])
    for documento in range(1, len(documentos)+1):
        sd = 0
        for lista_invertida in listas_invertidas:
            for i in lista_invertida:
                if (i[0] == documento): 
                    sd += i[1]
                    break
        if (sd != 0):
          heapq.heappush(r, (sd, documento))
    return heapq.nlargest(k, r)
 
# Exemplo
documentoPorVez("bolsonaro", indices, 10)  

[(24, 100),
 (18, 70),
 (16, 66),
 (14, 252),
 (11, 179),
 (11, 103),
 (11, 20),
 (10, 16),
 (9, 220),
 (9, 130)]

In [8]:
# Funcao que implementa a consulta termo-por-vez    
def termoPorVez(consulta, indices, k):
    a = {}
    listas_invertidas = []
    r = []
    for palavra in consulta.split(" "):
        if palavra in indices.keys():
            listas_invertidas.append(indices[palavra])
    for lista_invertida in listas_invertidas:
        for p in lista_invertida:
            d = p[0]
            freq = p[1]
            if (d in a.keys()):
                a[d] += freq
            else:
                a[d] = freq
    for (d, ad) in a.items():
        sd = ad
        heapq.heappush(r, (sd, d))
    return heapq.nlargest(k, r)
  
# Exemplo  
termoPorVez("bolsonaro", indices, 10)

[(24, 100),
 (18, 70),
 (16, 66),
 (14, 252),
 (11, 179),
 (11, 103),
 (11, 20),
 (10, 16),
 (9, 220),
 (9, 130)]

In [0]:
 # Definindo consultas
  
consultas = ["presidente", "educação", "privatização", "empresa", "bolsonaro"]

In [0]:
# Executando as consultas 

resultados_documento = []
resultados_termo = []
resultados_documento_tempo = []
resultados_termo_tempo = []
k = 10

for consulta in consultas:
  inicio_documento = time.time()
  resultados_documento.append(documentoPorVez(consulta, indices, k))
  fim_documento = time.time()
  resultados_documento_tempo.append(fim_documento - inicio_documento)
  
  inicio_termo = time.time()
  resultados_termo.append(termoPorVez(consulta, indices, k))
  fim_termo = time.time()
  resultados_termo_tempo.append(fim_termo - inicio_termo)


In [11]:
# Criando e imprimindo a tabela com os dados dos documentos

tabela_consultas = pd.DataFrame()
tabela_consultas['consulta'] = consultas
tabela_consultas['documento_por_vez'] = resultados_documento
tabela_consultas['termo_por_vez'] = resultados_termo
tabela_consultas['comparacao'] = tabela_consultas.documento_por_vez == tabela_consultas.termo_por_vez
tabela_consultas

,consulta,documento_por_vez,termo_por_vez,comparacao
0,presidente,"[(20, 16), (13, 140), (9, 138), (9, 130), (9, ...","[(20, 16), (13, 140), (9, 138), (9, 130), (9, ...",True
1,educação,"[(9, 218), (3, 195), (2, 177), (2, 89), (1, 25...","[(9, 218), (3, 195), (2, 177), (2, 89), (1, 25...",True
2,privatização,"[(1, 245), (1, 221), (1, 204), (1, 184), (1, 62)]","[(1, 245), (1, 221), (1, 204), (1, 184), (1, 62)]",True
3,empresa,"[(10, 177), (9, 107), (6, 262), (4, 95), (3, 2...","[(10, 177), (9, 107), (6, 262), (4, 95), (3, 2...",True
4,bolsonaro,"[(24, 100), (18, 70), (16, 66), (14, 252), (11...","[(24, 100), (18, 70), (16, 66), (14, 252), (11...",True


In [12]:
# Comparando os tempos medios de de execucao e uso de memoria

tabela_tempo = pd.DataFrame()
tabela_tempo['tempo_médio_documento_por_vez'] = resultados_documento_tempo
tabela_tempo['tempo_médio_termo_por_vez'] = resultados_termo_tempo
tabela_tempo

,tempo_médio_documento_por_vez,tempo_médio_termo_por_vez
0,0.002136,0.000161
1,0.000454,0.000031
2,0.000146,0.000007
3,0.000505,0.000034
4,0.001980,0.000139


# Questão 3

Implemente uma das versões de consulta conjuntiva (AND) (Fig. 5.20 ou 5.21).

1.  Defina 5 consultas conjuntivas (AND).
2.  Execute as 5 consultas em cada algoritmo retornando os top-10 documentos (parâmetro k do algoritmo).
3.  Dê evidências de que sua implementação está correta.

In [0]:
# Funcao que implementa a consulta conjuntiva (AND)

def consulta_and(consulta, indices, k):
  
    indices_consulta = []
    ranking = []
  
    for palavra in consulta.split(" "):
        if palavra in indices.keys():
            indices_consulta.append(indices[palavra])
  
    todas_as_consultas = [item for sublista in indices_consulta for item in sublista]
    todas_as_consultas.sort()
  
    for i in range(len(todas_as_consultas)):
        score_documento = 0
    d = todas_as_consultas.pop()
    for indice in todas_as_consultas:
        if indice[0] == d[0]:
            score_documento += indice[1] + d[1]
        if score_documento != 0:
            heapq.heappush(rank, (score_documento, d[0]))
        
    return heapq.nlargest(k, ranking)

In [0]:
# Definindo consultas

consultas = ["presidente", "educação", "privatização", "empresa", "bolsonaro"]

In [15]:
# Criando e imprimindo a tabela com os dados dos documentos

resultados_and = []
k = 10

for consulta in consultas:
    resultados_and.append(consulta_and(consulta, indices, k))


tabela_and = pd.DataFrame()
tabela_and['consulta'] = consultas
tabela_and['AND'] = resultados_and
tabela_and

,consulta,AND
0,presidente,[]
1,educação,[]
2,privatização,[]
3,empresa,[]
4,bolsonaro,[]
